In [75]:
# Libraries
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import requests
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler

from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os, re, struct, socket

import import_ipynb
import project_models

In [76]:
#GPU Checking
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 3060


Data Preprocessing

In [77]:
#IDS2017_Monday ==== No anomolies
#IDS2017_data = 'data\TrafficLabelling\Monday-WorkingHours.pcap_ISCX.csv'
#IDS2017_DF = pd.DataFrame(pd.read_csv(IDS2017_data))


#IDS2017_features = IDS2017_DF.iloc[:, [6, 1, 3, 2, 4, 5, 7, 84]] # 85 Columns
# Summing the total length of forward and backward packets
#IDS2017_features['Total Length'] = IDS2017_DF.iloc[:, 11] + IDS2017_DF.iloc[:, 12]
#IDS2017_features.head()

In [78]:
## KDDcup Corrected File
#KDD_fp = 'data\kddcup\kddcup_corrected.csv'
#KDD_DF = pd.DataFrame(pd.read_csv(KDD_fp))
#KDD_DF.head()

In [79]:
## PCAP DATA
pcap_fp = 'data\pcap\midnight.csv'
pcap_DF = pd.DataFrame(pd.read_csv(pcap_fp))
pcap_DF.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,23.63.110.66,39.62.1.38,SSL,1498,Continuation Data
1,2,0.000000,23.63.110.66,39.62.1.38,TCP,1494,[TCP Retransmission] 443 > 37488 [ACK] Seq=1...
2,3,0.000013,103.12.198.146,14.2.1.224,SSL,1446,Continuation Data
3,4,0.000015,103.12.198.146,14.2.1.224,TCP,1446,[TCP Retransmission] 443 > 44502 [ACK] Seq=1...
4,5,0.000016,103.12.198.146,14.2.1.224,TCP,1454,[TCP Retransmission] 443 > 44502 [ACK] Seq=1...


In [80]:
# Helper function to convert IP addresses into integers
def encode_ip(ip_address):
    try:
        return struct.unpack("!L", socket.inet_aton(ip_address))[0]
    except socket.error:
        # Handle the case for invalid IP addresses
        return 0


#Info Extracter for Pcap type files
def extract_network_info(info_str):
    # Dictionary to hold the extracted values
    extracted_info = {
        'sending_port': None,
        'receiving_port': None,
        'length': None,
        'seq': None,
        'ack': None,
        'win': None,
        'tsval': None,
        'tsecr': None,
        'sle_sre': [],
        'info_messages': [],
    }
    
    # Check for and extract port and length information
    match = re.search(r'(\S+)\s+\d+\s+(\d+)\s*>\s*(\d+)(?:.*Len=(\d+))?', info_str)
    if match:
        # Protocol and port information
        extracted_info['protocol'] = match.group(1)
        extracted_info['sending_port'] = int(match.group(2))
        extracted_info['receiving_port'] = int(match.group(3))
        
        # Length information if present
        if match.group(4):
            extracted_info['length'] = int(match.group(4))

    # Extract other information if present
    for key in ['seq', 'ack', 'win', 'tsval', 'tsecr']:
        match = re.search(r'{}=(\d+)'.format(key.capitalize()), info_str)
        if match:
            extracted_info[key] = int(match.group(1))

    # Extract SLE and SRE pairs
    extracted_info['sle_sre'] = re.findall(r'SLE=(\d+)\s*SRE=(\d+)', info_str)
    
    # Capture informational messages like "Ignored Unknown Record" or "Continuation Data"
    info_msg_match = re.search(r'\[.*?\]', info_str)
    if info_msg_match:
        extracted_info['info_messages'].append(info_msg_match.group(0))

    return extracted_info


Custom Dataset Construction

In [81]:
## The Dataset
class NetworkDataset(Dataset):
    def __init__(self, directory_path):
    # Load and concatenate all CSV files
        self.data = pd.concat(
            [pd.read_csv(os.path.join(directory_path, file)) for file in os.listdir(directory_path) if file.endswith('.csv')],
            ignore_index=True
        )

        # Sort data by 'Time' if necessary
        #self.data.sort_values(by='Time', inplace=True)
        
        # Preprocess data (fill missing values, extract features, encode data)
        self.data.fillna(self.get_default_fill_values(), inplace=True)
        self.extract_and_encode_features()

        #Scaling
        self.scaler = StandardScaler()
        self.scale_features()

    def get_default_fill_values(self):
        # Default fill values for missing data
        return {
            'latency': 0,
            'traffic_type': 'unknown',
            'anomaly_label': 0,
            'bottleneck': 'no',
            'Length': 0,
            'Source': '0.0.0.0',
            'Destination': '0.0.0.0',
            'Protocol': 'unknown',
            'Time': self.data['Time'].median() if not self.data['Time'].isna().all() else 0,
            'Info': 'No Information'
        }

    def extract_and_encode_features(self):
        # Extract and encode features
        self.data['Info'] = self.data['Info'].astype(str)
        extracted_info = self.data['Info'].apply(extract_network_info).apply(pd.Series)
        self.data = pd.concat([self.data, extracted_info], axis=1)
        self.data['source_encoded'] = self.data['Source'].apply(encode_ip)
        self.data['destination_encoded'] = self.data['Destination'].apply(encode_ip)
        self.data['Protocol'] = pd.factorize(self.data['Protocol'])[0]
        self.data['Time'] = self.data['Time'].astype(float)
        self.data['Length'] = self.data['Length'].astype(float)

        # Precompute BW & Throughput
        self.data['Time_diff'] = self.data['Time'].diff().fillna(0).clip(lower=1e-5)
        self.data['Bandwidth'] = self.data['Length'] / self.data['Time_diff']
        self.data['Throughput'] = self.data['Bandwidth']  # Simplified, adjust if different logic needed
        self.data['Latency'] = self.data['Length'] / 1000  # Assuming Length is in bytes and Latency is in ms

        # Type Conversions
        self.data['Time'] = self.data['Time'].astype(float)
        self.data['Length'] = self.data['Length'].astype(float)
        self.data['Bandwidth'] = self.data['Bandwidth'].astype(float)
        self.data['Throughput'] = self.data['Throughput'].astype(float)
        self.data['Latency'] = self.data['Latency'].astype(float)

    def scale_features(self):
        # Scale model input feautres
        features_to_scale = ['Time', 'Length', 'Time_diff', 'Bandwidth', 'Throughput', 'Latency']
        self.data[features_to_scale] = self.scaler.fit_transform(self.data[features_to_scale])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        features = torch.tensor([row['Time'], row['source_encoded'], row['destination_encoded'], row['Protocol'], row['Length']], dtype=torch.float32)

        ## Reg Features
        bandwidth = torch.tensor([row['Bandwidth']], dtype=torch.float32)
        throughput = torch.tensor([row['Throughput']], dtype=torch.float32)
        latency = torch.tensor([row['Latency']], dtype=torch.float32)
        # network_flow = Client -> Port -> Switch -> Master Swtich -> Firewall -> Inline Network Encryter -> Router -> Outbound                 
        # resource_usage = no RAM/CPU data available 


        ## Classification targets
        #traffic_type = torch.tensor(pd.factorize(data['traffic_type'])[0], dtype=torch.long)
        #anomaly_detection = torch.tensor(data['anomaly_label'].values, dtype=torch.long)
        #network_resource_status = torch.tensor(pd.factorize(data['resource_status'])[0], dtype=torch.long)
        #bottlenecks = torch.tensor(pd.factorize(data['bottleneck'])[0], dtype=torch.long)
        
        return {
    'features': features,
    'bandwidth': bandwidth,
    'throughput': throughput,
    'latency': latency,
    #'traffic_type': traffic_type,
    #'anomaly_detection': anomaly_detection,
    #'network_resource_status': network_resource_status,
    #'bottlenecks': bottlenecks,
    }


In [82]:
## Using the Dataset
batch_size = 256
dataset = NetworkDataset('data\pcap')
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Training and Test set splits
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=4)

In [83]:
for data in loader:
    features = data['features']
    bandwidth = data['bandwidth']
    throughput = data['throughput']
    latency = data['latency']

    print(features, bandwidth, throughput, latency)
    break

tensor([[-1.6850e+00,  3.9003e+08,  6.5837e+08,  0.0000e+00,  7.9314e-01],
        [-1.6850e+00,  3.9003e+08,  6.5837e+08,  1.0000e+00,  7.8796e-01],
        [-1.6850e+00,  1.7289e+09,  2.3501e+08,  0.0000e+00,  7.2573e-01],
        ...,
        [-1.6849e+00,  2.3514e+08,  1.7329e+09,  2.0000e+00, -1.0295e+00],
        [-1.6849e+00,  2.3514e+08,  1.7329e+09,  2.0000e+00, -1.0399e+00],
        [-1.6849e+00,  2.3501e+08,  1.7329e+09,  1.0000e+00, -1.0166e+00]]) tensor([[ 1.1065],
        [ 1.1010],
        [ 0.5733],
        [ 1.0346],
        [ 1.0457],
        [-0.2759],
        [ 1.1120],
        [ 1.1065],
        [ 1.1010],
        [-0.4949],
        [ 1.1120],
        [-0.4509],
        [ 0.8217],
        [-0.3485],
        [ 0.8272],
        [-0.9589],
        [-0.8565],
        [-0.8814],
        [-0.8510],
        [ 0.2755],
        [ 2.9976],
        [-0.9457],
        [-0.8731],
        [-0.8412],
        [ 0.8217],
        [ 0.8328],
        [-0.7222],
        [ 0.8217],
    

In [84]:
input_size = 5
hidden_size = 128
output_size = 1
dropout = 0.2
bidirectional = False

model = project_models.ntwkRNN(input_size, hidden_size, output_size, dropout, bidirectional).to(device)

epochs = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

rnn_tloss, rnn_vloss,rnn_vacc = project_models.train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device)

C:\Users\adams\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
#GRU
model = project_models.ntwkGRU(input_size, hidden_size, output_size, dropout, bidirectional).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

GRU_tloss, GRU_vloss,GRU_vacc = project_models.train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device)

In [ ]:
#LSTM
model = project_models.ntwkLSTM(input_size, hidden_size, output_size, dropout, bidirectional).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

lstm_tloss, lstm_vloss, lstm_vacc = project_models.train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device)

In [ ]:
#Transformer
model = project_models.ntwkPETransformer(input_size, hidden_size, output_size, num_layers = 5, nhead = 3).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)


Trans_tloss, Trans_vloss, Trans_vacc = project_models.train_Transformer(model,epochs, criterion, optimizer, train_loader, test_loader, device)